In [3]:
import requests
from bs4 import BeautifulSoup 
import pandas as pd
from pandas import Series
import numpy as np
import os
os.chdir("../Data")

# usert agent para evitar baneaos
headers = {
    "user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/71.0.3578.80 Chrome/71.0.3578.80 Safari/537.36"
}

# La url principal
url = 'https://www.tableandvine.com/main.asp?request=ARTICLES&article=14'
# peticion request y parseo de beautifulsoup
res = requests.get(url, headers=headers)
soup = BeautifulSoup(res.text, 'html.parser')

#seleccion del trozo de html a grandes rasgos que nos interesa 
tds=soup.find_all("td")
body=tds[17]
body

<td valign="top"><!--<SPAN CLASS="menu"> Pairing Food & Wine</SPAN><BR><BR><BR>--><div style="text-align: justify;"><span style="font-size: 18pt; font-weight: bold;"><span style="color: rgb(75, 0, 130);">Seven Tips For Pairing Food &amp; Wine</span></span></div><div style="text-align: justify;"><span style="font-weight: bold;"><br/></span></div><div style="text-align: justify;"><img align="right" alt="" border="0px" src="/pdf/pairingfoodwinesquare.jpg" style="margin: 5px 5px 5px 10px;"/><span style="font-size: 14pt;"><span style="font-weight: bold; font-size: 14pt;">Wine and food pairing is a balancing act, an art rather than an exact science.</span> You want to balance flavors, weight and intensity. A delicate dish should be matched with a subtle wine and a hearty meal with a sturdier wine. Wines can thus be chosen to complement the meal. For example, the apple and apricot flavors of a Riesling are an ideal complement for a pork roast (a dish often served with apple sauce or stuffed w

In [4]:
#Vamos a buscar los titulos de los textos que coindiden con las comidas que casan con vinos
all_results = body.select('div> div>span')[1:]
food= [x.text.strip() for x in all_results]
filter_food = list(filter(lambda x: x !="", food))
filter_food

['Beef',
 'Lamb',
 'BBQ',
 'Hamburger or Sausage',
 'Chicken',
 'Feathered Game (guinea fowl, pheasant, squab, etc.)',
 'Pasta',
 'Pasta with Tomato Sauce',
 'Fresh Water or Lighter Fish (trout, sole, etc.)',
 'Shellfish (clams, mussels, scallops, oysters, etc.)',
 'Lobster',
 'Salmon',
 'Smoked Salmon or other Smoked Fish',
 'Soft, Creamy Cheeses with washed rind (brie, camembert, etc.)',
 'Hard, Aged Cheeses (cheddar, aged gouda, manchego, etc.)',
 "Goat's Cheese"]

In [5]:
#ahora vamos a coger los textos de cada titulo , osea los vinos que coinciden con cada comida o titulo
all_results_2 = body.select('div>span')[12:]
content_clean= [x.text.strip() for x in all_results_2][6:]

filter_content = list(filter(lambda x: x !="", content_clean))
#¡¡¡no existe una patron!! debería ser comida, lista vino, comida ,lista d evinos...se incumple la linea 3 y 4
#que debería ir juntas. Así que vamos a juntarlas 
#y depues juntar:la lineas de vinos PREVIAS a 3 y 4 +  las lineas 3 y 4 (la 1) + las lineas TRAS 3 y 4 
filter_content

['Beef',
 'Cabernet Sauvignon, Merlot, Bordeaux, red Zinfandel, Rioja, Ribera del Duero, Châteauneuf-du-Pâpe, Hermitage, Côte-Rôtie, Australian Shiraz, Super Tuscan, Barbaresco or Barolo',
 'Lamb',
 'Bordeaux (especially Médoc), Rioja, Ribera del Duero, Châteauneuf-du-Pâpe, Barbaresco or Barolo Pork',
 'Riesling, Cru Beaujolais, Rioja, Côtes du Rhône or New World Chardonnay',
 'BBQ',
 'Côtes du Rhône, Châteauneuf-du-Pâpe, Shiraz, Petite Syrah or red Zinfandel',
 'Hamburger or Sausage',
 'Côtes du Rhône, Châteauneuf-du-Pâpe, Shiraz, Petite Syrah or red Zinfandel',
 'Chicken',
 'California or Australian Chardonnay, Riesling, dry Vouvray, white Burgundy, red Burgundy, Gigondas, Côtes du Rhône, Cabernet Sauvignon, Cabernet Franc, Merlot, red Zinfandel or Valpolicella',
 'Feathered Game (guinea fowl, pheasant, squab, etc.)',
 'Red Burgundy, Pinot Noir or Rioja',
 'Pasta',
 'Pinot Grigio, Vernaccia, Barbera, Dolcetto, Chianti, Pinot Noir or Cabernet Franc',
 'Pasta with Tomato Sauce',
 "Chia

In [6]:
#juntamos lineas 3 y 4 y arreglamos el patron 
lines_3_4=[filter_content[i]+filter_content[i+1] for i in [3,4]]
joined__lines_3_4=["".join(lines_3_4)]
#la linea de vinos antes de 3 y 4 :escla linea 1  y la selecionamos tambien 
line_1=[filter_content[1]]
#las lineas de vinos tras 3 y 4 : la linea de 8 en adelante que sean pares
content_for_pair=filter_content[8:]
lines_pairs_before_8=[content_for_pair[x] for x in range(0,len(content_for_pair),2)]
#unimos todas las lineas correspondientes a vinos
final_wines=line_1+joined__lines_3_4+lines_pairs_before_8
final_wines

['Cabernet Sauvignon, Merlot, Bordeaux, red Zinfandel, Rioja, Ribera del Duero, Châteauneuf-du-Pâpe, Hermitage, Côte-Rôtie, Australian Shiraz, Super Tuscan, Barbaresco or Barolo',
 'Bordeaux (especially Médoc), Rioja, Ribera del Duero, Châteauneuf-du-Pâpe, Barbaresco or Barolo PorkRiesling, Cru Beaujolais, Rioja, Côtes du Rhône or New World ChardonnayRiesling, Cru Beaujolais, Rioja, Côtes du Rhône or New World ChardonnayBBQ',
 'Côtes du Rhône, Châteauneuf-du-Pâpe, Shiraz, Petite Syrah or red Zinfandel',
 'California or Australian Chardonnay, Riesling, dry Vouvray, white Burgundy, red Burgundy, Gigondas, Côtes du Rhône, Cabernet Sauvignon, Cabernet Franc, Merlot, red Zinfandel or Valpolicella',
 'Red Burgundy, Pinot Noir or Rioja',
 'Pinot Grigio, Vernaccia, Barbera, Dolcetto, Chianti, Pinot Noir or Cabernet Franc',
 "Chianti, Morellino di Scansano, Salice Salentino or Montepulciano d'Arbruzzo",
 'White Bordeaux, Meursault or other good white Burgundy',
 'Rich Australian Chardonnay or S

In [7]:
#vamos a comprobar si el numero de titulos (comidas) coinciden con el numero de contenidos (vinos)
#de esta manera sabremos si hemos seleccionado todo 
len(final_wines)==len(filter_food)

True

In [8]:
#hacemos un zip para conseguir tuplas 
zip_food_wines=list(zip(filter_food, final_wines))

In [9]:
#lo pasamos a dataframe
data_pairing_not_explode=pd.DataFrame(zip_food_wines, columns=["food","wines"])

In [11]:
#esplotamos la columna wines

data_food_wines=pd.concat(   [Series(row['food'], row['wines'].split(','))  for _, row in data_pairing_not_explode.iterrows()]).reset_index()


In [12]:
#renombramos las columnas 
data_food_wines.columns=['Wines','Food']
data_food_wines.head(20)

,Wines,Food
0,Cabernet Sauvignon,Beef
1,Merlot,Beef
2,Bordeaux,Beef
3,red Zinfandel,Beef
4,Rioja,Beef
5,Ribera del Duero,Beef
6,Châteauneuf-du-Pâpe,Beef
7,Hermitage,Beef
8,Côte-Rôtie,Beef
9,Australian Shiraz,Beef


In [13]:
#guardamos limpio..... 
data_food_wines.to_csv("pairing.csv")